In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib.pyplot as plt
import joblib
from pathlib import Path

# sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, KFold, cross_validate, cross_val_score, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.base import BaseEstimator, TransformerMixin, clone

#hiperparamentros search
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

# Importações locais
from setup_notebook import setup_path
setup_path()
from src.model_utils import *
from src.preprocess_utils_tic import preprocessador_titanic

In [2]:
BASE = Path.cwd().parent   
# =====================================================
# ⚙️ 0. carregamento dos preprocessador 
# =====================================================
temp = joblib.load(BASE /'src'/'preprocess_Titanic_v1.0.joblib')
PP2=temp['preprocessador']

# # =====================================================
# # 📁 1. Leitura dos dados & Separação das bases
# # =====================================================

DATA_DIR = BASE / "data" / "processed"
X_train = pd.read_csv(DATA_DIR / "X_train_final.csv").reset_index(drop=True)
X_test  = pd.read_csv(DATA_DIR / "X_test_final.csv")
y_train = pd.read_csv(DATA_DIR / "y_train_final.csv").values.ravel()
y_test  = pd.read_csv(DATA_DIR / "y_test_final.csv")

In [3]:
# # =====================================================
# #  🤖 3.Definição dos Modelos Base
# # =====================================================
model_RF = RandomForestClassifier()
pipe_RF       = pipe_models(model_RF,PP2)

#primeira busca
# 1. Definição do Espaço de Busca (Grades de Parâmetros)
param_dist = {
    'model__n_estimators': randint(100, 500),
    'model__max_depth': [None, 5, 8, 10, 15, 20],
    'model__min_samples_split': randint(2, 20),
    'model__min_samples_leaf': randint(1, 20),
    'model__max_features': ['sqrt', 'log2', None],
    'model__bootstrap': [True, False]
}

# 2. Configuração do RandomSearch
# Usamos cv=5 ou 10 para otimizar tempo, e n_iter para definir quantas tentativas
random_search = RandomizedSearchCV(
    estimator=pipe_RF, 
    param_distributions=param_dist,
    n_iter=50,          # Testa 50 combinações aleatórias
    cv=10,              # Validação cruzada em cada tentativa
    scoring='accuracy', # Métrica alvo
    n_jobs=-1,          # Usa todos os núcleos do processador
    random_state=42,
    verbose=1
)

# 3. Execução da Busca
print("🔍 Iniciando a busca pelos melhores hiperparâmetros...")
random_search.fit(X_train, y_train)

# 4. Resultados
print(f"\n✅ Melhor Acurácia no CV: {random_search.best_score_*100:.2f}%")

print(f"📌 Melhores Parâmetros: {random_search.best_params_}")

random_search.best_estimator_

🔍 Iniciando a busca pelos melhores hiperparâmetros...
Fitting 10 folds for each of 50 candidates, totalling 500 fits

✅ Melhor Acurácia no CV: 83.64%
📌 Melhores Parâmetros: {'model__bootstrap': True, 'model__max_depth': 20, 'model__max_features': 'log2', 'model__min_samples_leaf': 3, 'model__min_samples_split': 8, 'model__n_estimators': 351}


Pipeline(steps=[('preprocess', preprocessador_titanic()),
                ('model',
                 RandomForestClassifier(max_depth=20, max_features='log2',
                                        min_samples_leaf=3, min_samples_split=8,
                                        n_estimators=351))])

In [5]:
# # =====================================================
# #  🤖 3.Definição dos Modelos Base
# # =====================================================
from scipy.stats import randint, uniform
model_RF = RandomForestClassifier()
pipe_RF       = pipe_models(model_RF,PP2)

#primeira busca
# 1. Definição do Espaço de Busca (Grades de Parâmetros)
param_dist = {
    'model__n_estimators': randint(120, 300),          # Focado perto dos 144 que venceram
    'model__max_depth': [7, 8, 9, 10],                 # Refinando perto do 8
    'model__min_samples_split': randint(9, 15),        # Focado perto do 11
    'model__min_samples_leaf': [2, 3, 4],              # Refinando perto do 2
    'model__max_features': [None],                     # Venceu como None, vamos manter fixo ou testar 'sqrt'
    'model__criterion': ['gini', 'entropy'],           # Testando formas de medir a pureza
    'model__bootstrap': [True],                        # Mantendo True já que RF depende disso
    'model__ccp_alpha': uniform(0, 0.02)               # Adicionando complexidade de poda
}

# 2. Configuração do RandomSearch
# Usamos cv=5 ou 10 para otimizar tempo, e n_iter para definir quantas tentativas
random_search = RandomizedSearchCV(
    estimator=pipe_RF, 
    param_distributions=param_dist,
    n_iter=250,          # Testa 100 combinações aleatórias
    cv=5,              # Validação cruzada em cada tentativa
    scoring='accuracy', # Métrica alvo
    n_jobs=-1,          # Usa todos os núcleos do processador
    random_state=42,
    verbose=1
)

# 3. Execução da Busca
print("🔍 Iniciando a busca pelos melhores hiperparâmetros...")
random_search.fit(X_train, y_train)

# 4. Resultados
print(f"\n✅ Melhor Acurácia no CV: {random_search.best_score_*100:.2f}%")
print(f"📌 Melhores Parâmetros: {random_search.best_params_}")
random_search.best_estimator_

🔍 Iniciando a busca pelos melhores hiperparâmetros...
Fitting 5 folds for each of 250 candidates, totalling 1250 fits

✅ Melhor Acurácia no CV: 82.82%
📌 Melhores Parâmetros: {'model__bootstrap': True, 'model__ccp_alpha': 0.01621106661563666, 'model__criterion': 'entropy', 'model__max_depth': 9, 'model__max_features': None, 'model__min_samples_leaf': 2, 'model__min_samples_split': 11, 'model__n_estimators': 204}


Pipeline(steps=[('preprocess', preprocessador_titanic()),
                ('model',
                 RandomForestClassifier(ccp_alpha=0.01621106661563666,
                                        criterion='entropy', max_depth=9,
                                        max_features=None, min_samples_leaf=2,
                                        min_samples_split=11,
                                        n_estimators=204))])